In [14]:
!pip install sentence-transformers transformers accelerate PyPDF2


In [15]:
import torch
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from PyPDF2 import PdfReader
import numpy as np


In [17]:
def load_pdf(path):
    reader = PdfReader(path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text

docs = load_pdf("AI_ML Engineering – Advanced Internship Tasks.pdf") 


In [18]:
def chunk_text(text, chunk_size=500, overlap=100):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i+chunk_size])
        chunks.append(chunk)
    return chunks

chunks = chunk_text(docs)
print(f"Total chunks: {len(chunks)}")


Total chunks: 2


In [19]:
embedder = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedder.encode(chunks, convert_to_numpy=True, show_progress_bar=True)
embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
def retrieve(query, k=3):
    q_emb = embedder.encode([query], convert_to_numpy=True)[0]
    q_emb = q_emb / np.linalg.norm(q_emb)
    scores = np.dot(embeddings, q_emb)
    top_k_idx = np.argsort(scores)[-k:][::-1]
    return [chunks[i] for i in top_k_idx]


In [21]:
# Load a small model (distilgpt2 is light and fast)
generator = pipeline("text-generation", model="distilgpt2", device=0 if torch.cuda.is_available() else -1)


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

C:\Users\kiran\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kiran\.cache\huggingface\hub\models--distilgpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better per

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [22]:
chat_history = []

def chatbot(query, history=chat_history, k=3):
    retrieved = retrieve(query, k)
    context = "\n\n".join(retrieved)

    prompt = f"""
Use the CONTEXT to answer the QUESTION.

CONTEXT:
{context}

HISTORY:
{history}

QUESTION:
{query}

Answer in a clear and simple way:
"""

    response = generator(prompt, max_length=250, num_return_sequences=1, do_sample=True)
    answer = response[0]['generated_text'].split("Answer in a clear and simple way:")[-1].strip()

    history.append((query, answer))
    return answer


In [23]:
print(chatbot("Summarize the document in 3 lines."))
print(chatbot("What is the main idea?"))


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=250) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=250) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/

: The above information is strictly for general purposes only.

What are the steps for the project?
This project has been running for six months, and the project is designed to give you more efficient, fast, and agile development. The goal for the project is to give you more efficient, fast, and agile development. The goal for the project is to give you more efficient, fast, and agile development. The goal for the project is to give you more efficient, fast, and agile development. The goal for the project is to give you more efficient, fast, and agile development. The goal for the project is to give you more efficient, fast, and agile development. The goal for the project is to give you more efficient, fast, and agile development. The goal for the project is to give you more efficient, faster, and agile development. The goal for the project is to give you more efficient, fast, and agile development. The goal for the project is to give you more efficient, fast, and agile development. Th